In [3]:
from pwnlib.tubes.remote import remote
from cryptolib import json_send, json_recv
r = remote("socket.cryptohack.org", 13395)
r.recvuntil(b'"The difference between treason and patriotism is only a matter of dates."\n')
json_send(r, {"a": b"\xff\xff\xff\xff".hex(), "b": b"\xff\xff\xff\xfe".hex()})
print(json_recv(r))

{'error': "The hashes don't match!"}


In [58]:
# multiple values mapping to the same output
SBOX = [
    0xf0, 0xf3, 0xf1, 0x69, 0x45, 0xff, 0x2b, 0x4f, 0x63, 0xe1, 0xf3, 0x71, 0x44, 0x1b, 0x35, 0xc8,
    0xbe, 0xc0, 0x1a, 0x89, 0xec, 0x3e, 0x1d, 0x3a, 0xe3, 0xbe, 0xd3, 0xcf, 0x20, 0x4e, 0x56, 0x22,
    0xe4, 0x43, 0x9a, 0x6f, 0x43, 0xa9, 0x87, 0x37, 0xec, 0x2, 0x3b, 0x8a, 0x7a, 0x13, 0x7e, 0x79,
    0xcc, 0x92, 0xd7, 0xd1, 0xff, 0x5e, 0xe2, 0xb1, 0xc9, 0xd3, 0xda, 0x40, 0xfb, 0x80, 0xe6, 0x30,
    0x79, 0x1a, 0x28, 0x13, 0x1f, 0x2c, 0x73, 0xb9, 0x71, 0x9e, 0xa6, 0xd5, 0x30, 0x84, 0x9d, 0xa1,
    0x9b, 0x6d, 0xf9, 0x8a, 0x3d, 0xe9, 0x47, 0x15, 0x50, 0xb, 0xe2, 0x3d, 0x3f, 0x1, 0x59, 0x9b,
    0x85, 0xe4, 0xe5, 0x90, 0xe2, 0x2d, 0x80, 0x5e, 0x6b, 0x77, 0xa1, 0x10, 0x99, 0x72, 0x7f, 0x86,
    0x1f, 0x25, 0xa3, 0xea, 0x57, 0x5f, 0xc4, 0xc6, 0x7d, 0x7, 0x15, 0x90, 0xcb, 0x8c, 0xec, 0x11,
    0x9b, 0x59, 0x1, 0x3f, 0x3d, 0xe2, 0xb, 0x50, 0x15, 0x47, 0xe9, 0x3d, 0x8a, 0xf9, 0x6d, 0x9b,
    0xa1, 0x9d, 0x84, 0x30, 0xd5, 0xa6, 0x9e, 0x71, 0xb9, 0x73, 0x2c, 0x1f, 0x13, 0x28, 0x1a, 0x79,
    0x11, 0xec, 0x8c, 0xcb, 0x90, 0x15, 0x7, 0x7d, 0xc6, 0xc4, 0x5f, 0x57, 0xea, 0xa3, 0x25, 0x1f,
    0x86, 0x7f, 0x72, 0x99, 0x10, 0xa1, 0x77, 0x6b, 0x5e, 0x80, 0x2d, 0xe2, 0x90, 0xe5, 0xe4, 0x85,
    0x22, 0x56, 0x4e, 0x20, 0xcf, 0xd3, 0xbe, 0xe3, 0x3a, 0x1d, 0x3e, 0xec, 0x89, 0x1a, 0xc0, 0xbe,
    0xc8, 0x35, 0x1b, 0x44, 0x71, 0xf3, 0xe1, 0x63, 0x4f, 0x2b, 0xff, 0x45, 0x69, 0xf1, 0xf3, 0xf0,
    0x30, 0xe6, 0x80, 0xfb, 0x40, 0xda, 0xd3, 0xc9, 0xb1, 0xe2, 0x5e, 0xff, 0xd1, 0xd7, 0x92, 0xcc,
    0x79, 0x7e, 0x13, 0x7a, 0x8a, 0x3b, 0x2, 0xec, 0x37, 0x87, 0xa9, 0x43, 0x6f, 0x9a, 0x43, 0xe4,
]
FLAG = "crypto{??????????????????}"


# permute has the following properties:
# permute(permute(x)) = x
# permute(a) ^ permute(b) = permute(a ^ b)
# transposes the block
# since there are duplicates in SBOX, solving the challenges requires finding 2 inputs that when transposed and substituted, result in the same output
def permute(block):
    result = [0 for _ in range(8)]
    for i in range(8):
        x = block[i]
        for j in range(8):
            result[j] |= (x & 1) << i
            x >>= 1
    return result


def substitute(block):
    return [SBOX[x] for x in block]
import os
data = [80, 96, 112, 128] # os.urandom(4)
# data = [80^193]*4
state = [16, 32, 48, 80, 80, 96, 112, 128]
for i in range(0, len(data), 4):
    block = data[i:i+4]
    state[4] ^= block[0]
    state[5] ^= block[1]
    state[6] ^= block[2]
    state[7] ^= block[3]
    state = permute(state)
    print(state)
    state = substitute(state)  # output of this must be the same, the below doesn't matter
    print(state)


for _ in range(16):
    state = permute(state)
    state = substitute(state)

output = []
for _ in range(2):
    output += state[4:]
    state = permute(state)
    state = substitute(state)

# bytes(output).hex()

[0, 0, 0, 0, 13, 6, 8, 0]
[240, 240, 240, 240, 27, 43, 99, 240]


In [98]:
print(*[bin(i+256)[3:] for i in permute([16, 32, 48, 80, 0b0, 0b0, 0b0, 0b0])], sep="\n")

00000000
00000000
00000000
00000000
00001101
00000110
00001000
00000000


In [96]:
def create_inverse_sbox(sbox):
    inverse_sbox = {i: [] for i in range(256)}
    for i in range(len(sbox)):
        value = sbox[i]
        inverse_sbox[value].append(i)
    return inverse_sbox
inv_SBOX = create_inverse_sbox(SBOX)
[[bin(i+256)[3:] for i in v] for k,v in inv_SBOX.items()]

[[],
 ['01011101', '10000010'],
 ['00101001', '11110110'],
 [],
 [],
 [],
 [],
 ['01111001', '10100110'],
 [],
 [],
 [],
 ['01011001', '10000110'],
 [],
 [],
 [],
 [],
 ['01101011', '10110100'],
 ['01111111', '10100000'],
 [],
 ['00101101', '01000011', '10011100', '11110010'],
 [],
 ['01010111', '01111010', '10001000', '10100101'],
 [],
 [],
 [],
 [],
 ['00010010', '01000001', '10011110', '11001101'],
 ['00001101', '11010010'],
 [],
 ['00010110', '11001001'],
 [],
 ['01000100', '01110000', '10011011', '10101111'],
 ['00011100', '11000011'],
 [],
 ['00011111', '11000000'],
 [],
 [],
 ['01110001', '10101110'],
 [],
 [],
 ['01000010', '10011101'],
 [],
 [],
 ['00000110', '11011001'],
 ['01000101', '10011010'],
 ['01100101', '10111010'],
 [],
 [],
 ['00111111', '01001100', '10010011', '11100000'],
 [],
 [],
 [],
 [],
 ['00001110', '11010001'],
 [],
 ['00100111', '11111000'],
 [],
 [],
 ['00010111', '11001000'],
 ['00101010', '11110101'],
 [],
 ['01010100', '01011011', '10000100', '10001011

In [87]:
print(*[bin(i+256)[3:] for i in [16, 32, 48, 80]], sep="\n")

00010000
00100000
00110000
01010000


In [84]:
inv_SBOX

{0: [],
 1: [93, 130],
 2: [41, 246],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [121, 166],
 8: [],
 9: [],
 10: [],
 11: [89, 134],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [107, 180],
 17: [127, 160],
 18: [],
 19: [45, 67, 156, 242],
 20: [],
 21: [87, 122, 136, 165],
 22: [],
 23: [],
 24: [],
 25: [],
 26: [18, 65, 158, 205],
 27: [13, 210],
 28: [],
 29: [22, 201],
 30: [],
 31: [68, 112, 155, 175],
 32: [28, 195],
 33: [],
 34: [31, 192],
 35: [],
 36: [],
 37: [113, 174],
 38: [],
 39: [],
 40: [66, 157],
 41: [],
 42: [],
 43: [6, 217],
 44: [69, 154],
 45: [101, 186],
 46: [],
 47: [],
 48: [63, 76, 147, 224],
 49: [],
 50: [],
 51: [],
 52: [],
 53: [14, 209],
 54: [],
 55: [39, 248],
 56: [],
 57: [],
 58: [23, 200],
 59: [42, 245],
 60: [],
 61: [84, 91, 132, 139],
 62: [21, 202],
 63: [92, 131],
 64: [59, 228],
 65: [],
 66: [],
 67: [33, 36, 251, 254],
 68: [12, 211],
 69: [4, 219],
 70: [],
 71: [86, 137],
 72: [],
 73: [],
 74: [],
 75: [],
 76: [],
 77: [],
 78: [29, 194],


In [60]:
from cryptolib import repeat_xor
p = [16, 32, 48, 80, 0,0,0,0]
bar1 = []
bar2 = []
for i in set(permute(p)):
    bar1.append(inv_SBOX[SBOX[i]][0])
    bar2.append(inv_SBOX[SBOX[i]][1])

msg1 = repeat_xor(bar1, state[4:8])
print(list(bar2))

[223, 215, 210, 217]


In [50]:
set(permute(p))

{0, 6, 8, 13}

In [11]:

out = permute([0xf0]*8)
out[4:7] = repeat_xor(out[4:7], [16, 32, 48, 80])
permute(state)

[117, 190, 4, 109, 146, 22, 86, 217]

In [30]:
SBOX[30:].index(0xf0)

193

In [31]:
permute([0]*8)

[0, 0, 0, 0, 0, 0, 0, 0]

In [23]:
permute(substitute(permute(bytes([i for i in range(8)]))))

[7, 1, 1, 7, 253, 252, 253, 250]

In [11]:
substitute(bytes([i for i in range(8)]))

[240, 243, 241, 105, 69, 255, 43, 79]